In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from time import time
tqdm.pandas()

C:\Users\Javi\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
basics = pd.read_csv('./title_principals.tsv', delimiter='\t', low_memory=False)

In [5]:
basics.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


##### We process information about cast and crew

In [3]:
principals_df = pd.read_csv('./title_principals.tsv', delimiter='\t', low_memory=False)
#crew_df = pd.read_csv('./title_crew.tsv', delimiter='\t', low_memory=False)

In [4]:
principals_df = principals_df[principals_df.category != 'self']
principals_df = principals_df.drop(columns=['job', 'characters', 'ordering'])

##### We want the actors and actresses in the same column

In [5]:
principals_df['category'] = principals_df.progress_apply(lambda row: 'actor/actress' if row.category in ['actor', 'actress'] else row['category'], axis=1)

100%|███████████████████████████████████████████████████████████████████| 35887579/35887579 [09:29<00:00, 63048.90it/s]


In [6]:
principals_df.head()

,tconst,nconst,category
1,tt0000001,nm0005690,director
2,tt0000001,nm0374658,cinematographer
3,tt0000002,nm0721526,director
4,tt0000002,nm1335271,composer
5,tt0000003,nm0721526,director


##### Check that every director and writer in the ```crew_df``` is also present in the ```principals_df``` (very long execution). Basically they are the exact same, so we can ignore ```crew_df```, and the code is commented so that it doesn't run every time. <br>
```
for index, row in crew_df.iterrows():
    
    print(f'Iteration {index}.') if index % 10000 == 0 else None
    
    if row.directors != '\\N':
        if principals_df[(principals_df.tconst == row.tconst) & (principals_df.nconst == row.directors)].empty:
            print(f'Directors {row.directors} not found')
    
    if row.writers != '\\N':
        if principals_df[(principals_df.tconst == row.tconst) & (principals_df.nconst == row.writers)].empty:
            print(f'Writers {row.writers} not found')
```

In [2]:
ratings_df = pd.read_csv('./title_ratings.tsv', delimiter='\t', low_memory=False)

##### We create a new column with the rating from 0 to 5 instead.

In [3]:
ratings_df['newAverageRating'] = ratings_df.averageRating/2
ratings_df = ratings_df.drop(columns=['averageRating'])

In [12]:
ratings_df.head()

,tconst,numVotes,newAverageRating
0,tt0000001,1688,2.85
1,tt0000002,208,3.00
2,tt0000003,1431,3.25
3,tt0000004,122,3.05
4,tt0000005,2229,3.05


#### Now we the creation of the final files

In [4]:
basics_df = pd.read_csv('./title_basics.tsv', delimiter='\t', low_memory=False)

In [5]:
# Cleaning of data and columns
only_movies_df = basics_df[(basics_df.titleType == 'movie') | (basics_df.titleType == 'tvMovie')]
only_movies_df = only_movies_df.drop(columns=['titleType'])

no_adult_df = only_movies_df.copy()
no_adult_df['isAdult'] = pd.to_numeric(only_movies_df.isAdult)
no_adult_df = no_adult_df[no_adult_df.isAdult == 0]
no_adult_df = no_adult_df.drop(columns=['isAdult'])

no_adult_df = no_adult_df.drop(columns=['endYear'])
no_adult_df = no_adult_df.replace('\\N', 'Unknown')

In [6]:
no_adult_df = pd.merge(no_adult_df, ratings_df, on=['tconst'])

In [7]:
no_adult_df.to_csv('preprocessed_movie_dataset.csv')

In [160]:
name_df = pd.read_csv('./name_basics.tsv', delimiter='\t', low_memory=False)

In [161]:
name_df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0053137,tt0072308,tt0031983,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0037382,tt0038355,tt0075213"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0054452,tt0057345,tt0056404"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0077975,tt0078723,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0069467,tt0050976,tt0050986"


#### Doesn't work. Too slow

In [75]:
categories = principals_df.category.unique().tolist()
# we get rid of categories "archive sound" and "archive footage"
categories = categories[:-2]
print(categories)

['director', 'cinematographer', 'composer', 'producer', 'editor', 'actor/actress', 'writer', 'production_designer']


In [150]:
new_df = pd.DataFrame()

new_df['tconst'] = principals_df.groupby('tconst', as_index=False).sum()['tconst']

for cat in categories:
    new_df[cat] = 'Unknown'
    
new_df.to_csv('cast_df.csv', index=False)

In [ ]:
aux_df = pd.read_csv('./cast_df.csv')

principals_idxs = principals_df.groupby('category').indices

cols = aux_df.columns.to_list()

for col in new_df.columns[1:]:
    
    print(f'Category "{col}"')

    for idx in tqdm(principals_idxs[col]):

        matching_idx = np.where(aux_df.to_numpy()[:, 0] == principals_df.to_numpy()[idx, 0])
        
        if aux_df.to_numpy()[matching_idx, cols.index(col)][0][0] == 'Unknown':   
            aux_df.to_numpy()[matching_idx, cols.index(col)] = principals_df.to_numpy()[idx, -2]
        else:
            aux_df.to_numpy()[matching_idx, cols.index(col)] += ',' + principals_df.to_numpy()[idx, -2]